In [1]:
import sys
#print(sys.path)
sys.path.insert(0, '/eos/user/j/jejarosl/.local/lib/python3.9/site-packages')

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

The scikit-learn version is 1.2.0.
Python version
3.9.12 (main, Jun  7 2022, 16:09:12) 
[GCC 11.2.0]
Version info.
sys.version_info(major=3, minor=9, micro=12, releaselevel='final', serial=0)


In [2]:
import time
import glob
import uproot
import numpy as np
import awkward as ak
import pickle
from tqdm import tqdm

import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt
from matplotlib import cm
import networkx as nx
import pandas as pd
from numba import jit

import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
import torch_geometric as pyg
from sklearn.manifold import TSNE
from sklearn import metrics

from sklearn.metrics import DetCurveDisplay, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import class_likelihood_ratios, precision_recall_fscore_support


%matplotlib inline
from matplotlib import rc
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

2024-03-05 10:43:01.073785: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 10:43:01.155574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
THRESHOLD_Z = 0.3

def mkdir_p(mypath):
    '''Function to create a new directory, if it not already exist.
       Args:
           mypath : directory path
    '''
    from errno import EEXIST
    from os import makedirs, path
    try:
        makedirs(mypath)
    except OSError as exc:
        if not (exc.errno == EEXIST and path.isdir(mypath)):
            raise

def computeEdgeAndLabels(track_data, edges, edges_labels, edge_features):
    '''Compute the truth graph'''
    print("Building the graph")

    n_tracks = len(track_data.gnn_pt)
    z_pca = np.array(track_data.gnn_z_pca)
    sim_vertex_ID = np.array(track_data.gnn_sim_vertex_ID)
    gnn_pt = np.array(track_data.gnn_pt)
    t_pi = np.array(track_data.gnn_t_pi)
    t_k = np.array(track_data.gnn_t_k)
    t_p = np.array(track_data.gnn_t_p)
    dz = np.array(track_data.gnn_dz)

    for i in tqdm(range(n_tracks)):
        z_diff = np.abs(z_pca[i+1:] - z_pca[i])
        ids = np.where(z_diff < THRESHOLD_Z)[0]
        indices = ids + i + 1

        edges.extend([(j, i) for j in indices])

        labels = (sim_vertex_ID[i] == sim_vertex_ID[indices]).astype(int)
        edges_labels.extend(labels)
        
        pt_diff = np.abs(gnn_pt[indices] - gnn_pt[i])
        time_comps = np.array([np.abs(t_pi[indices] - t_pi[i]),
                      np.abs(t_k[indices] - t_k[i]),
                      np.abs(t_p[indices] - t_p[i]),
                      
                      np.abs(t_pi[indices] - t_k[i]),
                      np.abs(t_pi[indices] - t_p[i]),
                      
                      np.abs(t_k[indices] - t_pi[i]),
                      np.abs(t_k[indices] - t_p[i]),
                      
                      np.abs(t_p[indices] - t_k[i]),
                      np.abs(t_p[indices] - t_pi[i])])
                      
        z_diff = z_diff[ids]
        
        for k, ind in enumerate(indices):
            dz_significance = (z_pca[i] - z_pca[ind]) / np.sqrt(dz[i]**2 + dz[ind]**2)
            edge_features.append([pt_diff[k], z_diff[k], dz_significance] + list(time_comps[:, k]))

In [4]:
def set_small_to_zero(a, eps=1e-8):
    a[np.abs(a) < eps] = 0
    return a

def remap_PIDs(pids):
    """
    Remaps particle IDs to a simplified classification scheme.
    
    Args:
    - pids (list): List of particle IDs to be remapped.
    
    Returns:
    - remapped_pids (list): List of remapped particle IDs.
    """
    # Mapping of particle IDs to simplified classification
    pid_map = {11: 0, 13: 0, 211: 0, 321: 1, 2212: 2, 3112: 2}
    
    # Remap PIDs using the pid_map dictionary
    remapped_pids = [pid_map.get(pid, -1) for pid in pids]
    
    return remapped_pids

def process_files(input_folder, output_folder, n_files=100000, offset=0):
    files = glob.glob(f"{input_folder}/*.root")
    print(f"Number of files: {len(files)}")

    X, Edges, Edges_labels, Edge_features = [], [], [], []
    PIDs_truth, Times_truth = [], []

    mkdir_p(output_folder)

    for i_file, file in enumerate(files[offset:offset+n_files]):
        
        print('\nProcessing file {} '.format(file))
        try:
            with uproot.open(file) as f:
                tree = f["vertices4DValid"]
                
                for ev, key in enumerate(tree):
                
                    t = tree[key]
                    track_data = t.arrays(["gnn_weight", "gnn_pt", "gnn_eta", "gnn_phi", "gnn_z_pca",
                                            "gnn_t_pi", "gnn_t_k", "gnn_t_p", "gnn_mva_qual", 'gnn_btlMatchChi2',
                                            'gnn_btlMatchTimeChi2', 'gnn_etlMatchChi2', "gnn_sim_vertex_ID",
                                            'gnn_etlMatchTimeChi2', 'gnn_pathLength', 'gnn_npixBarrel', 'gnn_npixEndcap',
                                            'gnn_mtdTime', 'gnn_is_matched_tp', 'gnn_dz', 'gnn_sigma_t0safe'])
                    truth_data = t.arrays(['gnn_tp_tsim', 'gnn_tp_tEst', 'gnn_tp_pdgId'])

                    number_of_tracks = len(track_data.gnn_weight)
                    print(f"{i_file}_{ev} : Have {number_of_tracks} tracks in the file")
                    
                    start = time.time()

                    x_ev = np.array([track_data.gnn_weight,
                                     track_data.gnn_pt,
                                     track_data.gnn_eta,
                                     track_data.gnn_phi,
                                     track_data.gnn_z_pca,
                                     track_data.gnn_t_pi,
                                     track_data.gnn_t_k,
                                     track_data.gnn_t_p,
                                     track_data.gnn_mva_qual,
                                     track_data.gnn_btlMatchChi2,
                                     track_data.gnn_btlMatchTimeChi2,
                                     track_data.gnn_etlMatchChi2,
                                     track_data.gnn_etlMatchTimeChi2,
                                     track_data.gnn_pathLength,
                                     track_data.gnn_npixBarrel,
                                     track_data.gnn_npixEndcap,
                                     track_data.gnn_mtdTime,
                                     track_data.gnn_dz,
                                     track_data.gnn_sigma_t0safe], 
                                    dtype=np.float32)

                    x_ev = set_small_to_zero(x_ev, eps=1e-5)

                    print(f"{i_file}_{ev} : Got the track properties")

                    X.append(x_ev)

                    edges, edges_labels, edge_features = [], [], []
                    pids, times = truth_data.gnn_tp_pdgId, truth_data.gnn_tp_tEst

                    # Call the function to compute edges and labels
                    computeEdgeAndLabels(track_data, edges, edges_labels, edge_features)
                    
                    print(len(edge_features), len(edge_features[0]))

                    ed_np = np.array(edges).T
                    Edges.append(ed_np)
                    PIDs_truth.append(pids)
                    Times_truth.append(times)
                    Edges_labels.append(edges_labels)
                    Edge_features.append(edge_features)
                    
                    
                    if (ev % 10 == 0 and ev != 0) or ev == len(tree.keys())-1:
                        stop = time.time()
                        print(f"t = {stop - start} ... Saving the pickle data for {i_file}_{ev}")

                        # Save the processed data into pickle files
                        with open(f"{output_folder}{i_file}_{ev}_node_features.pkl", "wb") as fp:
                            pickle.dump(X, fp)
                        with open(f"{output_folder}{i_file}_{ev}_edges.pkl", "wb") as fp:
                            pickle.dump(Edges, fp)
                        with open(f"{output_folder}{i_file}_{ev}_edges_labels.pkl", "wb") as fp:
                            pickle.dump(Edges_labels, fp)
                        with open(f"{output_folder}{i_file}_{ev}_edge_features.pkl", "wb") as fp:
                            pickle.dump(Edge_features, fp)
                        with open(f"{output_folder}{i_file}_{ev}_times_truth.pkl", "wb") as fp:
                            pickle.dump(Times_truth, fp)
                        with open(f"{output_folder}{i_file}_{ev}_PID_truth.pkl", "wb") as fp:
                            pickle.dump(PIDs_truth, fp)
                            
                        X, Edges, Edges_labels, Edge_features = [], [], [], []
                        PIDs_truth, Times_truth = [], []


        except Exception as e:
            print(f"Error: {e}")
            continue


    return number_of_tracks, Edges, PIDs_truth, Times_truth, Edges_labels, Edge_features

In [ ]:
input_folder = "/eos/user/k/kdeleo/Vertex_GNN_inputs/files_PU200/"
output_folder  = './dataset_tracks/'
number_of_tracks, Edges, PIDs_truth, Times_truth, Edges_labels, Edge_features = process_files(input_folder, output_folder)

Number of files: 13

Processing file /eos/user/k/kdeleo/Vertex_GNN_inputs/files_PU200/file_1876c212.root 
0_0 : Have 3339 tracks in the file
0_0 : Got the track properties
Building the graph


100%|██████████| 3339/3339 [00:02<00:00, 1226.12it/s]


245197 12
0_1 : Have 3179 tracks in the file
0_1 : Got the track properties
Building the graph


100%|██████████| 3179/3179 [00:02<00:00, 1140.55it/s]


249275 12
0_2 : Have 3212 tracks in the file
0_2 : Got the track properties
Building the graph


...

297151 12
0_29 : Have 2709 tracks in the file
0_29 : Got the track properties
Building the graph


100%|██████████| 2709/2709 [00:01<00:00, 1480.69it/s]


202839 12
0_30 : Have 3854 tracks in the file
0_30 : Got the track properties
Building the graph


100%|██████████| 3854/3854 [00:04<00:00, 868.05it/s] 


337127 12
t = 4.603540420532227 ... Saving the pickle data for 0_30


In [ ]:
print(f"Getting {Edges[0].shape[1]} edges")
print(f"For a fully connected graph we would have {number_of_tracks**2} edges")

In [ ]:
# plot distribution of labels

plt.hist(Edges_labels[0], bins=10, color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of false and true edges')

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Show plot
plt.show()

In [ ]:
plt.hist(Times_truth[0], bins=50, color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of times')

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)

# Show plot
# Set y-axis to log scale
plt.yscale('log')

plt.show()

In [ ]:
plt.hist(PIDs_truth[0], bins=50, color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of PIDs')

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)
# Set y-axis to log scale
plt.yscale('log')

# Show plot
plt.show()

In [ ]:
np.unique(PIDs_truth[0])

In [ ]:
pids_remaped = remap_PIDs(PIDs_truth[0])
plt.hist(pids_remaped, bins=10, color='skyblue', edgecolor='black')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of PIDs')

# Adding grid
plt.grid(True, linestyle='--', alpha=0.7)
# Set y-axis to log scale
plt.yscale('log')

# Show plot
plt.show()